In [15]:
from joblib import load
import pandas as pd
import uvicorn
from fastapi import FastAPI
from pydantic import BaseModel
from typing import Union

## Test API on local

In [3]:
predictor = load('./model/model_reg.pkl')
transformer = load('./model/transformer.pkl')

In [4]:
test_input = {'model_key':'Citroën', 
              'mileage':77334, 
              'engine_power':256, 
              'fuel':'diesel', 
              'paint_color':'black', 
              'car_type':'coupe', 
              'private_parking_available':True, 
               'has_gps':False, 
               'has_air_conditioning':True, 
               'automatic_car':False, 
               'has_getaround_connect':False, 
               'has_speed_regulator':True, 
               'winter_tires':False}

In [6]:
user_data = pd.DataFrame([test_input])
X = transformer.transform(user_data)
X_pred = predictor.predict(X)
print(f'The predicted price is {X_pred[0]}')

The predicted price is 179.22685972150043


### Deploy local API

In [16]:
class Features(BaseModel):
    model_key: str
    mileage: Union[int, float]
    engine_power: Union[int, float]
    fuel: str
    paint_color: str
    car_type: str
    private_parking_available: bool
    has_gps: bool
    has_air_conditioning: bool
    automatic_car: bool
    has_getaround_connect: bool
    has_speed_regulator: bool
    winter_tires: bool

C:\Users\romai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\pydantic\_internal\_fields.py:161: UserWarning: Field "model_key" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [13]:
app = FastAPI()

@app.post('/predict')
async def predict(input_data:Features):
    ''' Return predicted price according to car features'''
    predictor = load('./model/model_reg.pkl')
    transformer = load('./model/transformer.pkl')
    user_data = pd.DataFrame([input_data])
    X = transformer.transform(user_data)
    X_pred = predictor.predict(X)
    return {'The return predicted price is': X_pred[0]}